In [ ]:
!pip install nvidia-cudnn-cu11==8.6.0.163

# SETUP

In [ ]:
!pip install -r requirements.txt

In [10]:
from ai_experiments.Utils import utils
from ai_experiments.Lockout.helpers.VAE import VariationalAutoencoder, SupervisedVariationalAutoencoder
from ai_experiments.Lockout.helpers.Classifiers import ClassifierModel, ClassifierModelXL, convnext_pretrained, transfer_learning
from ai_experiments.Lockout.helpers.DataPreparation import CustomImageDataset, CustomImageDatasetLarge, create_data
from ai_experiments.Lockout.helpers.Inference import inference_loop, inference_loop_cls, compute_metrics_cls_for_threshold
from ai_experiments.Lockout.helpers.Train import train_loop_vae, train_loop_svae, train_loop_cls, train_loop_regression
# from ai_experiments.Lockout.helpers.Utils import save_images_wandb
import ai_experiments.Lockout.helpers.Clustering as c

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

ImportError: initialization failed

In [ ]:
# wandb.login()

In [ ]:
import numpy as np
import os
import glob
import re
from tqdm import tqdm
import pandas as pd

import torch
from torchvision import transforms
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F 
from torch import optim
import albumentations as A

from PIL import Image
import seaborn as sns 
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('/app/ai_experiments/Utils/sp_custom.mplstyle')
import matplotlib.patches as mpatches

from bson import json_util
import gc
from datetime import datetime

import ai_experiments.Utils.utils as utils
from signalpet.images import extract_bboxes_from_organs
import ast

gc.collect()

In [ ]:
###### torch.set_default_dtype(torch.float16)

current_time = datetime.now().strftime('%Y%m%d_%H%M%S')

class ConfigSettings:
    def __init__(self):
        self.TIME = current_time
        self.MODEL = 'Bladder-stone'
        self.EXP_NAME = f'{self.MODEL}_{current_time}'
        self.beta = 1.5
        self.alpha = 20
        self.IMG_DIR = '/mnt/westside-tools/training_data/'
        self.INPUT_DIM = 256
        self.IN_CHANNELS = 1
        self.NUM_CLS = 1
        self.split = 0.8
        self.filename = 'complete_df_info2'
        self.Z_DIM = 16
        self.PATCH_SIZE = 16,
        self.D_DIM = 768,
        self.NUM_EPOCHS = 100
        self.LR_RATE = 3e-4
        self.BATCH_SIZE = 32
        
        self.CLUSTERING = 'gm'
        self.k_means_params = {
            'n_clusters': 4,
            'init': 'k-means++', 
            'max_iter': 1000, 
            'tol': 0.0001, 
            'verbose': 0, 
            'random_state': 42, 
            'copy_x': True, 
            'algorithm': 'lloyd'
        }
        
CFG = ConfigSettings()

# run = wandb.init(
#   name= CFG.EXP_NAME, 
#   entity="lockout",
#   project="lockout-aina",
#   notes="train new model",
#   config=CFG,
#   job_type="training")

# Create Dataset 
to train the model you nead a dataframe with the meta information we store in mongo


you have two options:
- create your own dataset
- read from a .csv file (or train file) with the meta info 


In [2]:
class_name = "bladder-bladder-stone" #"fracture-limb-fracture"
mask_name = "bladder-stone-mask" #'fracture-limb'

## Create Dataset

In [5]:
from signalpet.basics import *
from signalpet.core import *
# from app.learning import *
# from app.core import *

q = """
    SELECT model_name, image_id, value_from, value_to, suggestion_source, suggestion_value, ai_accuracy, modifiediou_score from
    turing_queue_statistics
    where done_at > ('2023-06-22')
    and model_name = 'organ.%s'
    and channel in ('qc', 'qc-offline')
    and suggestion_source = 'ai'
    and ai_accuracy > 0.85
    and ai_accuracy < 1000
""" % (mask_name)
res = Druid().query(q)
df = pd.DataFrame(res)

utils.create_dataset(class_name, list(df['image_id']))

NameError: name 'utils' is not defined

In [ ]:
TRAININGSET_DIR = "/mnt/westside-tools/training_sets2/bladder-bladder-stone-20230914-134615" # EXAMPLE:'/mnt/westside-tools/training_sets2/' #CHANGE THIS DIRECTORY WITH THE ONE FROM THE STEP ABOVE
train = pd.read_pickle(os.path.join(TRAININGSET_DIR, "train.pkl"))
valid = pd.read_pickle(os.path.join(TRAININGSET_DIR, "valid.pkl"))

In [ ]:
# predictions - from mongo, to have the segmentation outline which we need for this particular problem

query_object_ids = utils.create_query(list(df['image_id']))
mongo = Mongo().col('images')
mongo_q = mongo.find(query_object_ids)

df_mongo = pd.DataFrame()
for row in mongo_q:

    bbox = extract_bboxes_from_organs(row, [mask_name], sources=["results"])[0]
    df_temp = pd.DataFrame({'meta': json.dumps(row, default=json_util.default), 
                            'ai_labels': row['results']['organs'][mask_name], 
                            'ai_labels_bbox': bbox,
                            '_id': row['_id']})
    df_mongo = pd.concat([df_mongo, df_temp])
    

In [ ]:
train._id = train._id.astype(str)
df_mongo['_id'] = df_mongo['_id'].astype(str)
combined_df = train.set_index('_id').merge(df_mongo.set_index('_id'), on='_id', how='inner')

complete_df_info = combined_df.merge(df, left_on='_id', right_on='image_id', how='inner')

# complete_df_info.to_csv(f'{class_name}_complete_df_info') # to save

In [ ]:
complete_df_info

In [ ]:
# Crop images
for idx, row in complete_df_info.iterrows():

    img = Image.open(f'{row["npyFilename"]}')
    (xmin, xmax, ymin, ymax) = ast.literal_eval(row["ai_labels_bbox"])
    # box - (left, upper, right, lower) int(x1), int(x2), int(y1), int(y2)
    bbox = (xmin, ymin, xmax, ymax)
    print(f'accuracy: {row["modifiediou_score"]}')
    
    try:
        crop_img = img.crop(bbox)
        crop_img.save(f'{row["npyFilename"].replace(".jpg", "-cropped.jpg")}')

        fig, ax = plt.subplots(1,2)
        ax[0].imshow(img)
        ax[1].imshow(crop_img)
        plt.show()
    
    except Exception as e:
        print(f'Error {e} with bbox: {bbox} \nimage_id: {row["image_id"]}')


In [ ]:
# complete_df_info[['ai_accuracy', 'modifiediou_score']] = complete_df_info[['ai_accuracy', 'modifiediou_score']].applymap(lambda x: round(np.float64(x), 3))

# # Create the scatter plot
# plt.figure(figsize=(10, 6))
# sns.scatterplot(x='ai_accuracy', y='modifiediou_score', data=complete_df_info)

# plt.title('AI Accuracy vs Modified IoU Score')
# plt.xlabel('AI Accuracy')
# plt.ylabel('Modified IoU Score')
# plt.grid(True)
# plt.show()

## Read df

In [ ]:
df_filename = f'{class_name}_complete_df_info'
complete_df_info = pd.read_csv(f"{df_filename}")
CFG.NUM_IMG = len(complete_df_info)
len(complete_df_info)

## Dataset

In [ ]:
confident_abnormal_df = complete_df_info[complete_df_info["ai_accuracy"]>0.85]
len(confident_abnormal_df)

meta = confident_abnormal_df # -- when we want to train only on correct crops
idx_train = np.round(CFG.NUM_IMG*CFG.split).astype(int)
print(CFG.NUM_IMG, idx_train)

In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((CFG.INPUT_DIM, CFG.INPUT_DIM)),  
    transforms.ToTensor(),  # Convert image to PyTorch Tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  
])  

augmentations = A.Compose([
    A.HorizontalFlip(p=0.3),
    A.VerticalFlip(p=0.3),
    A.RandomBrightnessContrast(p=0.2),
])

In [ ]:
# 1 channel images

train_dataset_a = CustomImageDataset(CFG.IMG_DIR, meta, 0, idx_train, transform=transform, augmentations=augmentations)
train_loader_a = DataLoader(train_dataset_a, batch_size=32, shuffle=True)

train_dataset = CustomImageDataset(CFG.IMG_DIR, meta, 0, idx_train, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = CustomImageDataset(CFG.IMG_DIR, meta, idx_train, CFG.NUM_IMG, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# 3 Channel images

train_dataset_large_a = CustomImageDatasetLarge(CFG.IMG_DIR, meta, 0, idx_train, transform=transform, augmentations=augmentations)
train_loader_large_a = DataLoader(train_dataset_large_a, batch_size=32, shuffle=False)

train_dataset_large = CustomImageDatasetLarge(CFG.IMG_DIR, meta, 0, idx_train, transform=transform, augmentations=augmentations)
train_loader_large = DataLoader(train_dataset_large, batch_size=32, shuffle=False)

test_dataset_large = CustomImageDatasetLarge(CFG.IMG_DIR, meta, idx_train, CFG.NUM_IMG, transform=transform)
test_loader_large = DataLoader(test_dataset_large, batch_size=32, shuffle=False)

In [ ]:
len(test_dataset), len(train_dataset)

# EMBED 


##  Data for visualizations

In [ ]:
# Extract the data from the dataset - (n_samples, n_features), 
height = CFG.INPUT_DIM
width = CFG.INPUT_DIM
channels = 1

X_train, y_true_train, y_true_train_binary = create_data(train_dataset)
X_test, y_true_test, y_true_test_binary = create_data(test_dataset)

X_train_img = X_train.reshape(-1, channels, CFG.INPUT_DIM, CFG.INPUT_DIM)
X_test_img = X_test.reshape(-1, channels, CFG.INPUT_DIM, CFG.INPUT_DIM)

test_image_ids = test_dataset.ids
train_image_ids = train_dataset.ids

# VAE

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VariationalAutoencoder(CFG.INPUT_DIM, CFG.Z_DIM).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=CFG.LR_RATE)
loss_fn = nn.BCELoss(reduction="sum")
beta = CFG.beta

model = train_loop_vae(model, optimizer, loss_fn, train_loader, CFG, DEVICE=DEVICE, wandb=wandb)

In [ ]:
torch.save(model.state_dict(), f'models/{CFG.EXP_NAME}.pth')
wandb.log({})

In [ ]:
# Load pretrained
# gc.collect()
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = VariationalAutoencoder(CFG.INPUT_DIM, CFG.Z_DIM).to(DEVICE)
# model.load_state_dict(torch.load('xxx.pth'))

### Inference 

In [ ]:
X_vae_train_recons, X_vae_train_z, y_vae_train = inference_loop(model, train_loader, DEVICE = DEVICE, v = True)
X_vae_test_recons, X_vae_test_z, y_vae_test = inference_loop(model, test_loader, DEVICE = DEVICE, v = True)

In [ ]:
clustering_model, out_of_sample_pred = c.cluster_img(X_vae_train_z, 
                                                     y_vae_train, 
                                                     method='kmeans', 
                                                     test_data=X_vae_test_z, 
                                                     test_lbl=y_vae_test, 
                                                     **CFG.k_means_params)

c.visualize_cluster(X_test_img, out_of_sample_pred, y_vae_test, num_images=8, name='embed')
c.visualize_cluster_dist(out_of_sample_pred, y_vae_test, plot_t='kde', name='embed')

y_vae_test_binary = np.where(y_vae_test>0.5,1, 0)
stats = c.cluster_info(out_of_sample_pred, y_vae_test_binary, v = True)

# CLASSIFIER 

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cls = ClassifierModel(CFG.INPUT_DIM).to(DEVICE)
cls_optimizer = optim.Adam(cls.parameters(), lr=CFG.LR_RATE)
cls_loss_fn = nn.BCELoss(reduction="sum")
cls = train_loop_cls(cls, cls_optimizer, cls_loss_fn, train_loader, CFG, DEVICE=DEVICE, wandb=wandb, verbose=True, th=0.5)

In [ ]:
torch.save(cls.state_dict(), f'models/{CFG.EXP_NAME}-classifier.pth')
wandb.log({})

In [ ]:
gc.collect()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cls = ClassifierModel(CFG.INPUT_DIM).to(DEVICE)
cls.load_state_dict(torch.load('models/Beta_20230814_193248_GM-classifier.pth'))

In [ ]:
# Train

X_test_cls_vec, y_pred_test_vec, y_test_cls_vec = inference_loop_cls(cls, test_loader, DEVICE = "cuda")
X_train_cls_vec, y_pred_train_vec, y_train_cls_vec = inference_loop_cls(cls, train_loader, DEVICE = "cuda")

In [ ]:
CFG.k_means_params = {
            'n_clusters': 10,
            'init': 'k-means++', 
            'max_iter': 10000, 
            'tol': 0.00001, 
            'verbose': 0, 
            'random_state': 42, 
            'copy_x': True, 
            'algorithm': 'lloyd'
        }

CFG.gm_params = {'n_components': 10,
     'covariance_type': 'full',
     'tol': 0.0001,
     'reg_covar': 1e-06,
     'max_iter': 1000,
     'n_init': 1,
     'init_params': 'kmeans',
     'weights_init': None,
     'means_init': None,
     'precisions_init': None,
     'random_state': 42}

wandb.log(CFG.__dict__)

In [ ]:
# cluster embeddings
clustering_model, out_of_sample_pred = c.cluster_img(X_train_cls_vec, 
                                                     y_train_cls_vec, 
                                                     method='gm', 
                                                     test_data = X_test_cls_vec,
                                                     test_lbl = y_test_cls_vec,
                                                     **CFG.gm_params)

c.visualize_cluster(X_test_img, out_of_sample_pred, y_test_cls_vec, num_images=3, name='embed')
c.visualize_cluster_dist(out_of_sample_pred, y_test_cls_vec, plot_t='kde', name='embed')

save_images_wandb(wandb)

In [ ]:
y_cls_test_binary = np.where(y_test_cls_vec>0.5,1, 0)
stats = c.cluster_info(out_of_sample_pred, y_cls_test_binary, y_test_cls_vec, v = True)

# ConvNextXL

## 1 CH

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = convnext_pretrained(in_chans=1, num_classes=CFG.NUM_CLS, depths=[3, 3, 27, 3], dims=[256, 512, 1024, 2048]).to(DEVICE)

In [ ]:
# torch.save(model.state_dict(), f'models/{CFG.EXP_NAME}-pretrainedConvNeXt.pth')
# wandb.log({})

In [ ]:
#Fine Tune
torch.cuda.empty_cache()
CFG.NUM_EPOCHS = 10
CFG.LR_RATE = 0.00001

wandb.log(CFG.__dict__)

optimizer = optim.AdamW(model.parameters(), lr=CFG.LR_RATE)
loss_fn = nn.CrossEntropyLoss()
model = train_loop_cls(model, optimizer, loss_fn, train_loader_a, CFG, DEVICE=DEVICE, wandb=wandb, verbose=False, th=0.5, use_clip_grad=True)

In [ ]:
torch.save(model.state_dict(), f'models/{CFG.EXP_NAME}-ConvNeXt.pth')
wandb.log({})

In [ ]:
# gc.collect()
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = convnext_pretrained(in_chans=1, num_classes=CFG.NUM_CLS, depths=[3, 3, 27, 3], dims=[256, 512, 1024, 2048]).to(DEVICE)
# model.load_state_dict(torch.load('models/convnext_20230815_233251-ConvNeXt.pth'))

In [ ]:
X_train_ConvNEXT_vec, y_pred_train_vec, y_train_ConvNEXT_vec = inference_loop_cls(model, train_loader, DEVICE = DEVICE)
X_test_ConvNEXT_vec, y_pred_test_vec, y_test_ConvNEXT_vec = inference_loop_cls(model, test_loader, DEVICE = DEVICE)

In [ ]:
compute_metrics_cls_for_threshold(y_train_ConvNEXT_vec, y_pred_train_vec, threshold=0.5)
compute_metrics_cls_for_threshold(y_test_ConvNEXT_vec, y_pred_test_vec, threshold=0.5)

In [ ]:
# cluster embeeddings
CFG.k_means_params = {
            'n_clusters': 6,
            'init': 'k-means++', 
            'max_iter': 10000, 
            'tol': 0.00001, 
            'verbose': 0, 
            'random_state': 42, 
            'copy_x': True, 
            'algorithm': 'lloyd'
        }

clustering_model, out_of_sample_pred = c.cluster_img(X_train_ConvNEXT_vec, 
                                                     y_train_ConvNEXT_vec, 
                                                     method='kmeans', 
                                                     test_data = X_test_ConvNEXT_vec,
                                                     test_lbl = y_test_ConvNEXT_vec,
                                                     **CFG.k_means_params)

c.visualize_cluster(X_test_img, out_of_sample_pred, y_test_ConvNEXT_vec, num_images=8, name='embed', display_all=False)
c.visualize_cluster_dist(out_of_sample_pred, y_test_ConvNEXT_vec, plot_t='kde', name='embed')
y_ConvNEXT_test_binary = np.where(y_test_ConvNEXT_vec>0.5,1, 0)
stats = c.cluster_info(out_of_sample_pred, y_ConvNEXT_test_binary, y_test_ConvNEXT_vec, v = True)

save_images_wandb(wandb)

In [ ]:
### Visualization tools for clustering

In [ ]:
stats = c.cluster_info(out_of_sample_pred, y_ConvNEXT_test_binary, y_test_ConvNEXT_vec, v = True)

In [ ]:
c.visualize_cluster(X_test_img, out_of_sample_pred, y_test_ConvNEXT_vec, num_images=8, name='embed', display_all=True)

In [ ]:
c.ss_cluster(X_test_img, labeled_data_test, test_image_ids, y_test_ConvNEXT_vec, out_of_sample_pred, num_images=8, name='cls3ch')

## ConvNexT 3 CH

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = convnext_pretrained(in_chans=3, num_classes=CFG.NUM_CLS, depths=[3, 3, 27, 3], dims=[256, 512, 1024, 2048]).to(DEVICE)

In [ ]:
# DEVICE

In [ ]:
#Fine Tune
torch.cuda.empty_cache()
CFG.NUM_EPOCHS = 10 #10 
CFG.LR_RATE = 0.0001 #perfect - 0.00001, slow

# wandb.log(CFG.__dict__)
model.train()
optimizer = optim.AdamW(model.parameters(), lr=CFG.LR_RATE)
loss_fn = nn.CrossEntropyLoss()
model = train_loop_cls(model, optimizer, loss_fn, train_loader_large_a, CFG, DEVICE=DEVICE, verbose=False, th=0.5, use_clip_grad=True)

model.eval()
torch.save(model.state_dict(), f'{CFG.EXP_NAME}-BladderC3-12.pth')
# wandb.log({})

In [ ]:
X_train_ConvNEXT_vec, y_pred_train_vec, y_train_ConvNEXT_vec = inference_loop_cls(model, train_loader_large, DEVICE = DEVICE)
X_test_ConvNEXT_vec, y_pred_test_vec, y_test_ConvNEXT_vec = inference_loop_cls(model, test_loader_large, DEVICE = DEVICE)

In [ ]:
X_test_ConvNEXT_vec

In [ ]:
# cluster embeeddings
CFG.k_means_params = {
            'n_clusters': 1,
            'init': 'k-means++', 
            'max_iter': 10000, 
            'tol': 0.00001, 
            'verbose': 0, 
            'random_state': 42, 
            'copy_x': True, 
            'algorithm': 'lloyd'
        }

clustering_model, out_of_sample_pred = c.cluster_img(X_train_ConvNEXT_vec, 
                                                     y_train_ConvNEXT_vec, 
                                                     method='kmeans', 
                                                     test_data = X_test_ConvNEXT_vec,
                                                     test_lbl = y_test_ConvNEXT_vec,
                                                     **CFG.k_means_params)

c.visualize_cluster(X_test_img, out_of_sample_pred, y_test_ConvNEXT_vec, num_images=8, name='embed', display_all=False)
c.visualize_cluster_dist(out_of_sample_pred, y_test_ConvNEXT_vec, plot_t='kde', name='embed')
y_ConvNEXT_test_binary = np.where(y_test_ConvNEXT_vec>0.5,1, 0)
stats = c.cluster_info(out_of_sample_pred, y_ConvNEXT_test_binary, y_test_ConvNEXT_vec, v = True)

# save_images_wandb(wandb)

In [ ]:
c.visualize_cluster(X_test_img, out_of_sample_pred, y_test_ConvNEXT_vec, num_images=8, name='embed', display_all=False)
c.visualize_cluster_dist(out_of_sample_pred, y_test_ConvNEXT_vec, plot_t='kde', name='embed')
y_ConvNEXT_test_binary = np.where(y_test_ConvNEXT_vec>0.5,1, 0)
stats = c.cluster_info(out_of_sample_pred, y_ConvNEXT_test_binary, y_test_ConvNEXT_vec, v = True)

In [ ]:
c.visualize_cluster(X_test_img, out_of_sample_pred, y_test_ConvNEXT_vec, num_images=8, name='embed', display_all=True)

In [ ]:
# c.ss_cluster(X_test_img, labeled_data_test, test_image_ids, y_test_ConvNEXT_vec, out_of_sample_pred, num_images=8, name='cls3ch')

# EXPORT TO ONNX

### CLUSTERING - SKLEARN MODEL

In [ ]:
import skl2onnx

clustering_model = 'insert your model' - load it

initial_type = [('inputs', skl2onnx.common.data_types.FloatTensorType([None, 2048]))]  # Replace 2 with the number of features in your actual model

Convert the scikit-learn model to ONNX format
onnx_model = skl2onnx.convert_sklearn(clustering_model, initial_types=initial_type)

# Save the ONNX model
with open(f"models/{CFG.EXP_NAME}-clustering_model.onnx", "wb") as f:
     f.write(onnx_model.SerializeToString())

### CLASSIFICATION

import torch.onnx

dynamic_axes = {'inputs': {0: 'batch_size'},  # indicating the dimensions that are dynamic
                'outputs': {0: 'batch_size'}}

x = torch.randn(32, 3, 256, 256).to(DEVICE) #B,C,H,W
torch.onnx.export(model, x, "model.onnx", verbose=False, input_names=['inputs'], output_names=['outputs'], dynamic_axes=dynamic_axes)